# 💠 CPRN CREATE · PHASE A2 · DG PROCESSING (OSM Guizhou Prov)

**贵州 CPRN 构建 · A2 基础路网与图生成**

`version`: `4.1.9`   
`author`: seika <seika@live.ca>   
`create_date`: `2025-11-19`   
`last_update`: `2025-11-20`   

---

### 本阶段目标
- 读取 A1 产出的贵州高速路段（OSM 清洗结果）
- 生成可计算路网所需的 LineString 集（含 length/端点 geohash）
- 构建基础 `DiGraph`，验证拓扑质量
- 准备持久化（Spatialite + Pickle），等待设施/门架数据接入

> 注：当前无门架/补丁道路，A2 聚焦单源 OSM，保留设施嵌入等 Hook 以便后续增量。


## 0. 环境配置 / Environment Setup


In [2]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import json
from pathlib import Path

import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
from loguru import logger
from shapely.geometry import Point

from cprn.model.geohash import Geohash

from taisl_sop.model.gdf.geometry import create_multi_crs_geometry
from taisl_sop.model.geometry.linestring.split import split_linestring_by_points

logger.info("✅ 环境加载完成")

2025-11-21 13:56:43.331 | INFO     | __main__:<module>:20 - ✅ 环境加载完成


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. 参数与路径 / Configurations

In [ ]:
VERSION_VENDOR = "v419"

# CRS
CRS_WGS84 = "epsg:4326"
CRS_CGCS2 = "epsg:4545"  # 与 A1 保持一致 / CGCS2000 3度带, 中心点为重庆东北方位

# Spatialite 扩展
PATH_SQLITE_EXT = Path("/opt/homebrew/Cellar/libspatialite/5.1.0/lib/mod_spatialite.dylib")

# 输入（A1 输出）
PATH_SQLITE_CPRN = Path("/Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite")
LAYER_NAME_RRL_OSM = "cprn_roads_osm_guizhou_v419"

# 输出（A2）
LAYER_NAME_CPRN_ROADS = "cprn_roads_cprn_osm_v419_guizhou"

PATH_FOLDER_CPRN_PAYLOAD = Path("/Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/CPRN_Guizhou_V419/")
CPRN_MODEL_NAME = "CPRN_DG_BASE_GZ_OSM_V419.pkl"

# 设施/门架占位（当前为空，保留 Hook）
MAX_SNAP_DIST = 500  # 设施与路网最近距离阈值
MAX_PROJ_DIST = 50   # 投影距离阈值

logger.info("输入: {}", PATH_SQLITE_CPRN)
logger.info("图层: {}", LAYER_NAME_RRL_OSM)
logger.info("输出: gdf_roads={}, dg_cprn={}", LAYER_NAME_CPRN_ROADS, CPRN_MODEL_NAME)


2025-11-21 13:57:37.906 | INFO     | __main__:<module>:24 - 输入: /Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite
2025-11-21 13:57:37.910 | INFO     | __main__:<module>:25 - 图层: cprn_roads_osm_guizhou_v419
2025-11-21 13:57:37.911 | INFO     | __main__:<module>:26 - 输出: gdf_roads=cprn_roads_cprn_osm_v419_guizhou, dg_cprn=CPRN_DG_BASE_GZ_OSM_V419.pkl


## 2. 数据载入与初探 / Load & Inspect

In [8]:
# 🌞 取回
from taisl_sop.data.base.spatialite import SpatialitePuller

sp = SpatialitePuller(
    path_sqlite=PATH_SQLITE_CPRN, 
    path_spatialite_ext=PATH_SQLITE_EXT, 
    set_precision=True, 
    decimal_precision=16, 
    driver='sqlite', 
    verbose=True)

# 构建 SQL 查询
sql = f"""
SELECT osm_id, road_code, rrl_name, 
       cls, rtype, maxspeed, oneway, layer,
       ASText(GEOMETRY) AS wkt
  FROM {LAYER_NAME_RRL_OSM};
  """

logger.info(f"🌞 从数据库取回: {LAYER_NAME_RRL_OSM}")
gdf_rrl = sp.query_sql_gdf(sql, col_geom=['wkt'])
gdf_rrl = gdf_rrl.drop(columns=['wkt'])
gdf_rrl = gdf_rrl.rename_geometry('geom')
gdf_rrl = gdf_rrl.set_geometry('geom')
gdf_rrl = gdf_rrl.set_crs(CRS_WGS84)

logger.info(f"✅ 数据取回完成")
logger.info(f"   表名: {LAYER_NAME_RRL_OSM}")
logger.info(f"   记录数: {len(gdf_rrl)}")
logger.info(f"   字段: {gdf_rrl.columns.tolist()}")

# 显示样本数据
gdf_rrl.sample(4)

2025-11-21 13:57:42.523 | INFO     | taisl_sop.data.base.spatialite:__init__:42 - SpatialitePuller initiated with path_sqlite: /Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite
2025-11-21 13:57:42.523 | INFO     | taisl_sop.data.base.spatialite:__init__:43 - Path_spatialite_ext: /opt/homebrew/Cellar/libspatialite/5.1.0/lib/mod_spatialite.dylib
2025-11-21 13:57:42.524 | INFO     | taisl_sop.data.base.spatialite:__init__:45 - Set_precision: ✅, decimal_precision: 16
2025-11-21 13:57:42.524 | INFO     | __main__:<module>:20 - 🌞 从数据库取回: cprn_roads_osm_guizhou_v419
2025-11-21 13:57:42.638 | INFO     | taisl_sop.data.base.spatialite:_setup_precision:83 - SpatiaLite version: 5.1.0
2025-11-21 13:57:42.639 | INFO     | taisl_sop.data.base.spatialite:_setup_precision:92 - ✅ Decimal precision set to: 16
2025-11-21 13:57:43.019 | INFO     | taisl_sop.data.geometry.handler:_handle_multiple_geometries:98 - Processed geometry columns: ['geom_wkt']
2025-11-21 13:57:43.020 | INFO    

Memory usage of df puller : gdf 2.73 MB, df None MB


,osm_id,road_code,rrl_name,cls,rtype,maxspeed,oneway,layer,geom
31437,866702381,,,BDG,ramp,0,F,1,"LINESTRING (106.96696 26.17098, 106.96685 26.1..."
30960,832180426,G75,兰海高速,BDG,NA,0,F,1,"LINESTRING (106.83151 28.19651, 106.83201 28.1..."
24635,598782067,S62,余册高速,MR,NA,0,F,0,"LINESTRING (107.32039 25.84926, 107.32096 25.8..."
32858,912740993,S09,兴义环城高速,BDG,NA,80,F,1,"LINESTRING (105.01346 25.0821, 105.0146 25.0848)"


### 2.1 快速体检 / Quick EDA

In [9]:
gdf_rrl.groupby(['oneway']).size()

oneway
B      304
F    35512
T       12
dtype: int64

## 3. 道路预处理 / Prepare Base Roads

In [10]:
# calculate length of roadline section by geom in cgcs2000 projection
gdf_rrl['length'] = gdf_rrl['geom'].to_crs(CRS_CGCS2).length.round(2)

In [11]:
# generate src/tgt point and vtx code from geom
# using '-00000' represent Z of null elevation in GeohashZ
gdf_rrl['geom_src_pt'] = gdf_rrl['geom'].apply(lambda x: Point(x.coords[0]))
gdf_rrl['geom_tgt_pt'] = gdf_rrl['geom'].apply(lambda x: Point(x.coords[-1]))
gdf_rrl['ghz_src_vtx'] = gdf_rrl['geom_src_pt'].apply(lambda x: Geohash.pt_encode(x, precision=12, upper=True)+'-00000')
gdf_rrl['ghz_tgt_vtx'] = gdf_rrl['geom_tgt_pt'].apply(lambda x: Geohash.pt_encode(x, precision=12, upper=True)+'-00000')

In [12]:
# create list of tuple of edges (src, tgt, attr)
lst_tup_edges = gdf_rrl.apply(
    lambda row: (
        row['ghz_src_vtx'],
        row['ghz_tgt_vtx'],
        {'osm_id': row['osm_id'],
         'road_code': row['road_code'],
         'knd': '',
         'cls': row['cls'],
         'rtype': row['rtype'],
         'maxspeed': row['maxspeed'],
         'oneway': row['oneway'],
         'layer': row['layer'],
         'weight': row['length']
        }),axis=1
        ).tolist()
lst_dct_edges = [(src, tgt, attr) for src, tgt, attr in lst_tup_edges if src != tgt]

## 4. 构建 CPRN 图 / Build Base DiGraph


In [13]:
## CREATE DG
DG_CPRN = nx.DiGraph()
DG_CPRN.add_edges_from(lst_dct_edges)

print(f"`DG_CPRN` (BASE Model) Created : {DG_CPRN}")

`DG_CPRN` (BASE Model) Created : DiGraph with 33546 nodes and 35752 edges


## 5. 图质量检验 / Graph QA


In [14]:
from collections import Counter


def describe_degree(dg: nx.DiGraph) -> pd.DataFrame:
    degrees = [(node, dg.in_degree(node), dg.out_degree(node)) for node in dg.nodes]
    df_deg = pd.DataFrame(degrees, columns=['node', 'in_deg', 'out_deg'])
    df_deg['total_deg'] = df_deg['in_deg'] + df_deg['out_deg']
    return df_deg.describe()


def connected_components_stats(dg: nx.DiGraph) -> dict:
    undirected = dg.to_undirected()
    components = list(nx.connected_components(undirected))
    sizes = [len(c) for c in components]
    return {
        'component_count': len(components),
        'max_component': max(sizes) if sizes else 0,
        'min_component': min(sizes) if sizes else 0,
        'size_distribution': Counter(sizes),
    }

df_degree_desc = describe_degree(DG_CPRN)
comp_stats = connected_components_stats(DG_CPRN)

logger.info("度统计:\n{}", df_degree_desc)
logger.info("连通分量: {}", comp_stats['component_count'])

2025-11-21 13:58:08.867 | INFO     | __main__:<module>:25 - 度统计:
             in_deg      out_deg     total_deg
count  33546.000000  33546.00000  33546.000000
mean       1.065760      1.06576      2.131521
std        0.305898      0.30658      0.423970
min        0.000000      0.00000      1.000000
25%        1.000000      1.00000      2.000000
50%        1.000000      1.00000      2.000000
75%        1.000000      1.00000      2.000000
max        3.000000      3.00000      5.000000
2025-11-21 13:58:08.868 | INFO     | __main__:<module>:26 - 连通分量: 10


In [12]:
## CREATE GDF of CPRN (based on gh of vtx and edge)
from taisl_sop.model.procs.dg2gdf import DiGraphToGeoDF

gdf_edges_cprn = DiGraphToGeoDF.to_gdf(DG_CPRN, geohash_decode_z=True)
gdf_edges_cprn.drop(columns=['geom_src', 'geom_tgt'], inplace=True)
gdf_edges_cprn = gdf_edges_cprn.rename(columns={'from':'vtx_src', 'to':'vtx_tgt'})

print(len(gdf_edges_cprn))
gdf_edges_cprn.sample(4)

35752


,vtx_src,vtx_tgt,oneway,layer,road_code,rtype,cls,osm_id,maxspeed,knd,weight,geom_edge
1979,WKE150QTZCCW-00000,WKE150KTEQ9U-00000,F,0,,ramp,,1259693175,0,,769.50,"LINESTRING Z (105.60985 25.49065 0, 105.60691 ..."
20890,WKEYY6RHH1C7-00000,WKEYY6TSEB9U-00000,F,3,G60;G69;G6002,ramp,BDG,167925514,0,,285.71,"LINESTRING Z (106.80788 26.51208 0, 106.80581 ..."
7786,WKS4ESX7GFS3-00000,WKS4EVEE8BDR-00000,F,0,S62,NA,MR,598781972,0,,855.08,"LINESTRING Z (107.03893 25.77733 0, 107.04464 ..."
857,WKUPTCR06JF2-00000,WKUPW0W8249Q-00000,F,-1,S14,NA,TUN,1123174235,0,,1104.29,"LINESTRING Z (107.13739 28.04405 0, 107.1476 2..."


## 6. 持久化 Hook / Persistence Hooks


### 🌖 PERSISTENCE : DG_BASE

In [17]:
from cprn.data.pickle import PickleIO

print (CPRN_MODEL_NAME)
PickleIO.dump_as_pickle(DG_CPRN, PATH_FOLDER_CPRN_PAYLOAD / CPRN_MODEL_NAME, compress=True)

CPRN_DG_BASE_GZ_OSM_V419.pkl
File renamed from /Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/CPRN_DG_BASE_GZ_OSM_V419_251121_temp.tar.gz to /Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/CPRN_DG_BASE_GZ_OSM_V419_251121_4678b74b5f9be79cc393e694a15192748a04adf89f0bf23e31c212d58e705988.tar.gz
Compressed pickle file saved as: /Users/seika/sync/lab_works/taisl/cprn_payload/cprn_models/CPRN_DG_BASE_GZ_OSM_V419_251121_4678b74b5f9be79cc393e694a15192748a04adf89f0bf23e31c212d58e705988.tar.gz


### 🌖 PERSISTENCE : DG_BASE to GDF

In [27]:
### 🌖 PERSISTENCE : gdf_roads_cprn 持久化（带原始几何）

# 准备持久化字段
cols_persist = ['osm_id', 'geom_edge', # 核心字段
        'vtx_src', 'vtx_tgt', 'weight', # 图关联字段
        'road_code', 'cls', 'rtype', # 道路属性
        'oneway', 'layer', 'maxspeed',
        ]

# 定义数据层名称
LAYER_NAME_CPRN_ROADS_CPRN = 'CPRN_dg_roads_osmbase_' + VERSION_VENDOR + '_guizhou'
print (LAYER_NAME_CPRN_ROADS_CPRN)

# 持久化
gdf_roads_cprn_persist = gdf_edges_cprn[cols_persist].copy()
gdf_roads_cprn_persist = gdf_roads_cprn_persist.set_geometry('geom_edge').set_crs(CRS_WGS84)

gdf_roads_cprn_persist.to_file(PATH_SQLITE_CPRN, driver='SQLite', 
    spatialite=True, layer=LAYER_NAME_CPRN_ROADS_CPRN, overwrite=True, 
    index=False, mode='w', encoding='utf-8')

print(f"✅ gdf_roads_cprn 已持久化：{LAYER_NAME_CPRN_ROADS_CPRN}")
print(f"    包含 {len(gdf_roads_cprn_persist)} 条边，{len(cols_persist)} 个字段")

CPRN_dg_roads_osmbase_v419_guizhou
✅ gdf_roads_cprn 已持久化：CPRN_dg_roads_osmbase_v419_guizhou
    包含 35752 条边，11 个字段


## 7. 下一步 / Next Steps

- [ ] 结合 EDA 输出，确认是否需要进一步断开道路或补充外部数据
- [ ] 如后续获取门架/设施数据，可直接在“设施 Hook”处追加投影逻辑
- [ ] 通过示例最短路验证 (e.g., 贵阳 ⇄ 遵义)
- [ ] 运行持久化单元，生成 Spatialite + Pickle 产物
